In [1]:
import os

if os.getcwd().endswith("notebooks"):
    os.chdir("..")
    print("using project root as working dir")

using project root as working dir


In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [3]:
import torch

device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
print(f"using {device} device")

using cuda device


In [5]:
import numpy as np
from datetime import datetime
from src.evaluator import Evaluator
from src.args import gridsearch_args

experiment = dict(
    key=datetime.now().strftime("%d-%m--%H-%M"),
    reps=5,
    args=gridsearch_args()
)
result = dict()

run_len = len(experiment['args']) * experiment['reps']
for i, args in enumerate(experiment["args"]):
    test_scores = []
    for rep in range(experiment["reps"]):
        print(f"running evaluator {i * experiment['reps'] + rep}/{run_len}")
        evaluator = Evaluator(
            args=args,
            writer_log_dir=f"new_runs/{experiment['key']}/s{args.epoch_graph_size}-a{args.epoch_graph_alpha}-b{args.epoch_graph_boredom_pth}-l{args.layers}xs{args.layer_size}--{rep}",
            device=device
        )
        evaluator.train(
            optimizer=torch.optim.Adam(evaluator.net.parameters(), lr=1e-3),
            save_fig=False
        )
        _, test_ap, _ = evaluator.test(
            epoch=args.epochs,
            save_fig=False
        )
        test_scores.append(test_ap)
        #evaluator.eval(args.epochs)

    result[f"{args.epoch_graph_size}-a{args.epoch_graph_alpha}-b{args.epoch_graph_boredom_pth}"] = np.mean(test_scores)

# print result
result = sorted(result.items(), key=lambda item: item[1])

running evaluator 0/45


building evaluator:   0%|          | 0/10 [00:00<?, ?it/s]

training model...: 0it [00:00, ?it/s]

testing model:   0%|          | 0/5 [00:00<?, ?it/s]

running evaluator 1/45


...

testing model:   0%|          | 0/5 [00:00<?, ?it/s]

running evaluator 44/45


building evaluator:   0%|          | 0/10 [00:00<?, ?it/s]

training model...: 0it [00:00, ?it/s]

testing model:   0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
print(*result, sep='\n')
#print(*result_by_loss, sep='\n')

('50-a1.0-b0.8', 0.18979737311601638)
('50-a0-b0.8', 0.2338776782155037)
('100-a1.0-b0.8', 0.7023717347532511)
('50-a0.5-b0.8', 0.7873981118202209)
('250-a1.0-b0.8', 0.9409003496170044)
('100-a0.5-b0.8', 0.9493442296981811)
('250-a0.5-b0.8', 0.9778411030769348)
('250-a0-b0.8', 0.9829733610153198)
('100-a0-b0.8', 0.9840125799179077)


In [27]:
result_by_loss

[('s250-a0.5-b0.8', 0.005878783667045862), ('s250-a1.0-b0.8', 0.0063466853293487795), ('s250-a0-b0.8', 0.02875102132548103), ('s100-a0.5-b0.8', 0.032567257878304924), ('s50-a0.5-b0.8', 0.03713049828155187), ('s100-a1.0-b0.8', 0.038373470434774494), ('s50-a1.0-b0.8', 0.054015376333496734), ('s100-a0-b0.8', 0.06140156823062557), ('s50-a0-b0.8', 0.0982753255891617)]

{'50-a1.0-b0.8': 0.18979737311601638,
 '50-a0-b0.8': 0.2338776782155037,
 '100-a1.0-b0.8': 0.7023717347532511,
 '50-a0.5-b0.8': 0.7873981118202209,
 '250-a1.0-b0.8': 0.9409003496170044,
 '100-a0.5-b0.8': 0.9493442296981811,
 '250-a0.5-b0.8': 0.9778411030769348,
 '250-a0-b0.8': 0.9829733610153198,
 '100-a0-b0.8': 0.9840125799179077}